In [ ]:
import scanpy as sc
import anndata as ad
import squidpy as sq
import numpy as np
import pandas as pd
import os
from glob import glob
import skmisc # install skmisc
import harmonypy # install harmonypy
import louvain # pip install leidenalg

In [ ]:
# Get the list of all files and directories
path = '/Users/nima.taherpour/Downloads/spaceranger-outs-data/'
path_list = os.path.join(path,'*')
data_list = sorted(glob(path_list))
print(len(data_list))
library__ids = ['/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_158_D1/spatial/tissue_hires_image.png','/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_159_C1/spatial/tissue_hires_image.png','/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_304_A1/spatial/tissue_hires_image.png','/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_304_C1/spatial/tissue_hires_image.png']
print(len(library__ids))

In [ ]:
rr3brainobject =list()
for i in range(0,len(data_list)):
    ob = sc.read_visium(data_list[i],count_file='filtered_feature_bc_matrix.h5',library_id='lid',load_images = True)
    ob.var_names_make_unique()
    rr3brainobject.append(ob)
    

print(rr3brainobject)

In [ ]:
paths = ['/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_158_D1/spatial/tissue_positions_list.csv','/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_159_C1/spatial/tissue_positions_list.csv','/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_304_A1/spatial/tissue_positions_list.csv','/Users/nima.taherpour/Downloads/spaceranger-outs-data/Sample_304_C1/spatial/tissue_positions_list.csv']
full = list()
for i in range(0,len(paths)):
    ob = pd.read_csv(paths[i],names = ['barcode','in_tissue','array_row','array_col','pxl_row_in_fullres','pxl_col_in_fullres'])
    ob = pd.DataFrame(ob)
    ob = ob.loc[ob['in_tissue'] == 1]
    ob.set_index('barcode',inplace = True)
    full.append(ob)

In [ ]:
path2 = '/Users/nima.taherpour/Downloads/metadata_rr3brain.xlsx'
brain_metadata = pd.read_excel(path2)
print(brain_metadata)

In [ ]:
rr3brainobjectup = rr3brainobject
for i in range(0,len(brain_metadata)):
    rr3brainobjectup[i].obs['section.name'] = brain_metadata['section.name'][i]
    rr3brainobjectup[i].obs['sample_condition'] = brain_metadata['sample_condition'][i]
    rr3brainobjectup[i].obs['mouse.id'] = brain_metadata['mouse.id'][i]
    rr3brainobjectup[i].obs['pxl_row_in_fullres'] = full[i]['pxl_row_in_fullres']
    rr3brainobjectup[i].obs['pxl_col_in_fullres'] = full[i]['pxl_col_in_fullres']

In [ ]:
rr3brainobjectrm = rr3brainobjectup
for i in range(0,len(rr3brainobjectrm)):
    rr3brainobjectrm[i].var['mt'] = rr3brainobjectrm[i].var_names.str.startswith('mt-')
    rr3brainobjectrm[i].var['rb1'] = rr3brainobjectrm[i].var_names.str.startswith('Rps')
    rr3brainobjectrm[i].var['rb2'] = rr3brainobjectrm[i].var_names.str.startswith('Rpl')
    continue
print(np.where(rr3brainobjectrm[3].var['mt'])[0])

In [ ]:
mito_genes = list()
rb1_genes = list()
rb2_genes = list()
for i in range(0,len(rr3brainobjectrm)):
    ob1 = rr3brainobjectrm[i].var_names.str.startswith('mt-')
    ob2 = rr3brainobjectrm[i].var_names.str.startswith('Rpl')
    ob3 = rr3brainobjectrm[i].var_names.str.startswith('Rps')
    mito_genes.append(ob1)
    rb1_genes.append(ob2)
    rb2_genes.append(ob3)
print(len(np.where(mito_genes[2])[0]))
print(len(np.where(rb1_genes[2])[0]))
print(len(np.where(rb2_genes[2])[0]))

In [ ]:
for i in range(0,len(rr3brainobjectrm)):
    mitoo_genes = mito_genes[i]
    rbb1_genes = rb1_genes[i]
    rbb2_genes = rb2_genes[i]
    remove = np.add(rbb1_genes,mitoo_genes)
    remove = np.add(remove,rbb2_genes)
    keep = np.invert(remove)
    rr3brainobjectrm[i] = rr3brainobjectrm[i][:,keep]
print(rr3brainobjectrm)

In [ ]:
rr3brainfiltered = rr3brainobjectrm
for i in range(0,len(rr3brainfiltered)):
    sc.pp.filter_cells(rr3brainfiltered[i], min_counts = 100)
    sc.pp.filter_cells(rr3brainfiltered[i],min_genes = 200)
    sc.pp.filter_genes(rr3brainfiltered[i],min_counts = 1)
print(rr3brainfiltered)

In [ ]:
Joined_rr3brain = ad.concat(rr3brainfiltered, axis = 0,join = 'outer')
print(Joined_rr3brain)

In [ ]:
print(Joined_rr3brain)

In [ ]:
sc.pp.highly_variable_genes(Joined_rr3brain,n_top_genes = 3000, flavor = 'seurat_v3')
sc.pp.regress_out(Joined_rr3brain,keys = 'n_genes')
sc.pp.scale(Joined_rr3brain, max_value = 10)


In [ ]:
Joined_rr3brain = Joined_rr3brain[:, Joined_rr3brain.var.highly_variable]

In [ ]:
sc.pp.pca(Joined_rr3brain,n_comps= 50,use_highly_variable = True)


In [ ]:
sc.pp.neighbors(Joined_rr3brain,n_neighbors = 23, n_pcs = 30,knn = True, use_rep = 'X_pca')
sc.tl.umap(Joined_rr3brain,min_dist = 0.30, n_components = 30) 

In [ ]:
sc.external.pp.harmony_integrate(Joined_rr3brain, key = ['sample_condition','section.name'])

In [ ]:
sc.pp.neighbors(Joined_rr3brain,n_neighbors = 23, n_pcs = 30,knn = True, use_rep = 'X_pca_harmony',key_added = 'neighborsafterharmony')
sc.tl.louvain(Joined_rr3brain,resolution = 0.60, neighbors_key = 'neighborsafterharmony')
sc.tl.umap(Joined_rr3brain,min_dist = 0.30, n_components = 30, neighbors_key = 'neighborsafterharmony')

In [ ]:
plotUMAP = sc.pl.umap(Joined_rr3brain,color = 'louvain')
print(plotUMAP)

In [ ]:

sc.tl.rank_genes_groups(Joined_rr3brain,groupby = 'louvain', groups = 'all', reference = 'rest',key_added = 'cluster_markers', method = 'wilcoxon',corr_method = 'benjamini-hochberg')
    

In [ ]:

sc.tl.rank_genes_groups(Joined_rr3brain,groupby = 'sample_condition', groups = 'all', reference = 'rest',key_added = 'flightvsground', method = 'wilcoxon' ,corr_method = 'benjamini-hochberg')

In [ ]:

flight_markers = sc.get.rank_genes_groups_df(Joined_rr3brain,group = 'flight',key = 'flightvsground', pval_cutoff = 0.05, log2fc_min = -10, log2fc_max = 10)
print(flight_markers)

In [ ]:

flight_markersinc = flight_markers.drop(flight_markers[(flight_markers.logfoldchanges < 0)].index)
flight_markersred = flight_markers.drop(flight_markers[(flight_markers.logfoldchanges > 0)].index)
flight_markersinc.head()

In [ ]:
path2 = '/Users/nima.taherpour/Downloads/genes.zip (Unzipped Files)/genes/updated_hearts_brains_INC_genes_only.csv'
path3 =  '/Users/nima.taherpour/Downloads/genes.zip (Unzipped Files)/genes/updated_hearts_brains_RED_genes_only.csv'
incgenes = pd.read_csv(path2)
redgenes = pd.read_csv(path3)
print(incgenes)

In [ ]:
overlap= pd.merge(flight_markersinc,incgenes, how = 'inner', left_on='names', right_on = 'x')
print(overlap)

In [ ]:
overlap1= pd.merge(flight_markersred,redgenes, how = 'inner', left_on='names', right_on = 'x')
print(overlap1)